<a href="https://www.kaggle.com/code/chamodkalupahana/simple-dnn-model-cham-cirrhosis?scriptVersionId=154032193" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e26/sample_submission.csv
/kaggle/input/playground-series-s3e26/train.csv
/kaggle/input/playground-series-s3e26/test.csv


### Load in data

In [2]:
training_dataset = pd.read_csv("/kaggle/input/playground-series-s3e26/train.csv")
test_dataset = pd.read_csv("/kaggle/input/playground-series-s3e26/test.csv")
training_dataset.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [3]:
training_dataset.describe()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000
mean,3952.000000,2030.173308,18373.146490,2.594485,350.561923,3.548323,83.902846,1816.745250,114.604602,115.340164,265.228969,10.629462,3.032511
std,2282.121272,1094.233744,3679.958739,3.812960,195.379344,0.346171,75.899266,1903.750657,48.790945,52.530402,87.465579,0.781735,0.866511
min,0.000000,41.000000,9598.000000,0.300000,120.000000,1.960000,4.000000,289.000000,26.350000,33.000000,62.000000,9.000000,1.000000
25%,1976.000000,1230.000000,15574.000000,0.700000,248.000000,3.350000,39.000000,834.000000,75.950000,84.000000,211.000000,10.000000,2.000000
50%,3952.000000,1831.000000,18713.000000,1.100000,298.000000,3.580000,63.000000,1181.000000,108.500000,104.000000,265.000000,10.600000,3.000000
75%,5928.000000,2689.000000,20684.000000,3.000000,390.000000,3.770000,102.000000,1857.000000,137.950000,139.000000,316.000000,11.000000,4.000000
max,7904.000000,4795.000000,28650.000000,28.000000,1775.000000,4.640000,588.000000,13862.400000,457.250000,598.000000,563.000000,18.000000,4.000000


In [4]:
x = training_dataset

### Convert categorical data to numberical

#### Replace values with binary

In [5]:
print('Possible Drug values: ',set(x['Drug']))
print('Possible Sex values: ',set(x['Sex']))
print('Possible Ascites values: ',set(x['Ascites']))
print('Possible Hepatomegaly values: ',set(x['Hepatomegaly']))
print('Possible Spiders values: ',set(x['Spiders']))
print('Possible Edema values: ',set(x['Edema']))
print('Possible Status values: ',set(x['Status']))

Possible Drug values:  {'D-penicillamine', 'Placebo'}
Possible Sex values:  {'F', 'M'}
Possible Ascites values:  {'N', 'Y'}
Possible Hepatomegaly values:  {'N', 'Y'}
Possible Spiders values:  {'N', 'Y'}
Possible Edema values:  {'S', 'N', 'Y'}
Possible Status values:  {'D', 'CL', 'C'}


In [6]:
x['Drug'].replace('D-penicillamine', 0, inplace=True)
x['Drug'].replace('Placebo', 1, inplace=True)

x['Sex'].replace('F', 0, inplace=True)
x['Sex'].replace('M', 1, inplace=True)

x['Ascites'].replace('N', 0, inplace=True)
x['Ascites'].replace('Y', 1, inplace=True)

x['Hepatomegaly'].replace('N', 0, inplace=True)
x['Hepatomegaly'].replace('S', 1, inplace=True)
x['Hepatomegaly'].replace('Y', 2, inplace=True)

x['Spiders'].replace('N', 0, inplace=True)
x['Spiders'].replace('Y', 1, inplace=True)

x['Edema'].replace('N', 0, inplace=True)
x['Edema'].replace('S', 1, inplace=True)
x['Edema'].replace('Y', 2, inplace=True)

x.head()


,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,0,21532,1,0,0,0,0,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,1,19237,0,0,0,0,0,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,1,13727,0,0,2,1,2,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,1,18460,0,0,0,0,0,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,1,16658,0,0,2,0,0,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


### Create submission

In [7]:
submission = pd.DataFrame()
submission['id'] = test_dataset['id']

submission['Status_C'] = 0.5
submission['Status_CL'] = 0.5
submission['Status_D'] = 0.5
submission.head()

,id,Status_C,Status_CL,Status_D
0,7905,0.5,0.5,0.5
1,7906,0.5,0.5,0.5
2,7907,0.5,0.5,0.5
3,7908,0.5,0.5,0.5
4,7909,0.5,0.5,0.5


In [8]:
submission.to_csv('/kaggle/working/submission.csv',index=False,header=True)